In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Read CSV file into a Pandas DataFrame
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Identify target variables and feature variables
# Target variables
target = df["IS_SUCCESSFUL"]

# Feature variables (drop EIN and NAME columns)
features = df.drop(columns=["EIN", "NAME", "IS_SUCCESSFUL"])

# Determine the number of unique values for each column
unique_counts = features.nunique()

# For columns with more than 10 unique values,
# determine number of data points for each unique value
for column in unique_counts.index:
    if unique_counts[column] > 10:
        print("Column:", column)
        print(features[column].value_counts())
        print("--------------------------------------------------")

# Based on unique value counts,pick a cutoff point to categorical variables
cutoff = 100

# Binning "rare" categorical variables together in a new value "Other"
for column in unique_counts.index:
    if unique_counts[column] > 10 and features[column].dtype == "object":
        counts = features[column].value_counts()
        rare_values = counts[counts < cutoff].index
        features[column] = features[column].apply(lambda x: "Other" if x in rare_values else x)

# Check if the binning was successful
for column in unique_counts.index:
    if unique_counts[column] > 10:
        print("Column:", column)
        print(features[column].value_counts())
        print("--------------------------------------------------")

# Encode categorical variables using pd.get_dummies()
features_encoded = pd.get_dummies(features)

# Split preprocessed data into features array (X) and target array (y)
X = features_encoded.values
y = target.values

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scale training and testing features datasets
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Column: APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64
--------------------------------------------------
Column: CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64
--------------------------------------------------
Column: ASK_AMT
5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64
--------------------------------------------------
Column: APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5 

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# Create a neural network model
model = Sequential()

# Add the first hidden layer
model.add(Dense(units=64, activation='relu', input_dim=X_train_scaled.shape[1]))

# Add a second hidden layer (optional)
# model.add(Dense(units=32, activation='relu'))

# Add output layer
model.add(Dense(units=1, activation='sigmoid'))

# Check structure of the model
model.summary()

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback to save the model's weights every five epochs
checkpoint = ModelCheckpoint("model_weights.h5", save_weights_only=True, save_best_only=True, monitor='val_loss', mode='min', verbose=1, period=5)

# Train model
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=50,
    batch_size=256,
    validation_data=(X_test_scaled, y_test),
    callbacks=[checkpoint]
)

# Evaluate model using test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model to an HDF5 file
model.save("AlphabetSoupCharity.h5")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 3329 (13.00 KB)
Trainable params: 3329 (13.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/50
101/101 [==============================] - 2s 6ms/step - loss: 0.6420 - accuracy: 0.6776 - val_loss: 0.5802 - val_accuracy: 0.7237
Epoch 2/50
101/101 [==============================] - 0s 5ms/step - loss: 0.5659 - accuracy: 0.7245 - val_loss: 0.5671 - val_accuracy: 0.7250
Epoch 3/50
101/101 [==============================] - 0s 4ms/step - loss: 0.5580 - accuracy: 0.7271 - val_loss: 0.5606 - val_accuracy: 0.7245
Epoch 4/50
101/101 [==============================] - 0s 4ms/step - loss: 0.5539 - accuracy: 0.7294 - val_loss: 0.5570 - val_accuracy: 0.7262
Epoch 5/50
 91/101 [==========================>...] - ETA: 0s - loss: 0.5525 - accuracy: 0.7288
Epoch 5: val_loss improved from inf to 0.55479, saving model to model_weights.h5
101/101 [==============================] - 0s 4ms/step - loss: 0.5517 - accuracy: 0.7297 - val_loss: 0.5548 - val_accuracy: 0.7272
Epoch 6/50
101/101 [==============================] - 0s 3ms/step - loss: 0.5504 - accuracy: 0.7289 - val_loss: 0.5548 - val

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:

# Create a neural network model
model = Sequential()

# Add first hidden layer with more neurons
model.add(Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Add second hidden layer with more neurons (optional)
model.add(Dense(units=64, activation='relu'))

# Add third hidden layer with more neurons (optional)
model.add(Dense(units=32, activation='relu'))

# Add output layer
model.add(Dense(units=1, activation='sigmoid'))

# Check structure of model
model.summary()

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Create a callback to save the model's weights every five epochs
checkpoint = ModelCheckpoint("model_weights.h5", save_weights_only=True, save_best_only=True, monitor='val_loss', mode='min', verbose=1, period=5)

# Train model w/ increased epochs
history = model.fit(
    X_train_scaled,
    y_train,
    epochs=100,  # Increase the number of epochs
    batch_size=256,
    validation_data=(X_test_scaled, y_test),
    callbacks=[checkpoint]
)

# Evaluate model using test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Save the model to an HDF5 file
model.save("AlphabetSoupCharity_optimized.h5")


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               6528      
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 16897 (66.00 KB)
Trainable params: 16897 (66.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/100
101/101 [==============================] - 2s 6ms/step - loss: 0.5794 - accuracy: 0.7100 - val_loss: 0.5577 - val_accuracy: 0.7250
Epoch 2/100
101/101 [==============================] - 0s 4ms/step - loss: 0.5532 - accuracy: 0.7291 - val_loss: 0.5534 - val_accuracy: 0.7319
Epoch 3/100
101/101 [==============================] - 0s 4ms/step - loss: 0.5494 - accuracy: 0.7308 - val_loss: 0.5546 - val_accuracy: 0.7241
Epoch 4/100
101/101 [==============================] - 0s 4ms/step - loss: 0.5477 - accuracy: 0.7321 - val_loss: 0.5511 - val_accuracy: 0.7280
Epoch 5/100
 97/101 [===========================>..] - ETA: 0s - loss: 0.5467 - accuracy: 0.7309
Epoch 5: val_loss improved from inf to 0.55005, saving model to model_weights.h5
101/101 [==============================] - 0s 4ms/step - loss: 0.5463 - accuracy: 0.7316 - val_loss: 0.5500 - val_accuracy: 0.7285
Epoch 6/100
101/101 [==============================] - 1s 5ms/step - loss: 0.5440 - accuracy: 0.7336 - val_loss: 0.5494